In [1]:
import numpy as np
from itertools import permutations, repeat, product
import gym
from libr.envs.cliff_walking2 import CliffWalkingEnv

In [2]:
env = CliffWalkingEnv()

In [ ]:
def onPolicyMCJC(env, numEpisodes, discount, epsilon, lenEp, tol = 10**(-4)):
    '''This is the main function for computing on-policy (epsilon-soft policy) Monte Carlo method.
    Inputs - (i) env: OpenAI gym environment
            (ii) numEpisodes: number of episodes
            (ii) discount: the discount factor for infinite horizon discounted DP
            (iv) epsilon: 
            (v) lenEp:
            (vi) tol: tolerance for stopping the iterations; the default value is 10^(-4)
    Outputs - Value and policy  
    For example, see Figure 5.6 of Sutton and Barto for the pseudocode
    '''
    #initialize
    #Q = np.zeros([env.nS, env.nA])
    Q=np.zeros([env.observation_space.n,env.action_space.n])
    returnCountFromEpisodes = np.zeros_like(Q)
    returnSumFromEpisodes = np.zeros_like(Q)
    policy = np.zeros([env.observation_space.n,1])
    sumRewardFromEpisode = 0
    
    def epsilonSoft(state, Q, epsilon, nA):
        pol = np.zeros([env.action_space.n])
        aStar = np.argmax(Q[state])
        for a in range(nA): #considering A(s) = nA for all s
            pol[a] = epsilon/nA
        pol[aStar] += 1-epsilon
        return pol

    for ep in range(numEpisodes):
        episode = [] # start with an empty episode
        currState = env.reset()
        for t in range(lenEp):
            epPolicy = epsilonSoft(currState,Q,epsilon,env.action_space.n)
        
            #randomly choose an action with prob assigned by epPolicy 
            currAction = np.random.choice(range(env.action_space.n),1, p=epPolicy)[0] 
        
            #generate a sample
            nextState, reward, done, _ = env.step(currAction)
        
            #add the sample to the episode
            episode.append([currState, currAction, reward])
        
            if done:
                break
            currState = nextState
        
        #lenEpisode = sum(1 for x in episode) # get the length of episode 
        #stateEp = np.zeros([lenEpisode])
        #actionEp = np.zeros([lenEpisode])
        stateEp = np.zeros([len(episode)])
        actionEp = np.zeros([len(episode)])
        
        for elem in range(len(episode)):
            stateEp[elem] = episode[elem][0]
            actionEp[elem] = episode[elem][1]
        #li = [list(zip(stateEp, p)) for p in permutations(actionEp)] #list of lists
        li = product(range(env.nS), range(env.nA))
        flat_list = [item for sublist in li for item in sublist] #flatten the list of lists into one list
        sa_list_of_tuples = set(flat_list) #retain unique elements of the list
        sa_list_of_lists = [list(elem) for elem in sa_list_of_tuples]
        
        for state,action in sa_list_of_lists:
            first_occ_step = next(idx for idx,x in enumerate(episode) if x[0] == state and x[1] == action)
            
            #return following first occurence of state, action
            first_occ_return = sum((discount**t)*x[2] for t,x in enumerate(episode[first_occ_step:]))
            
            sumRewardFromEpisode += first_occ_return
            returnSumFromEpisodes[state][action] = sumRewardFromEpisode #total return of (s,a) from all episodes
            returnCountFromEpisodes[state][action] += 1 #total number of occurences of (s,a) in all episodes
            Q[state,action] = returnSumFromEpisodes/returnCountFromEpisodes #average
    policy = np.argmax(Q,1)        
    return Q, policy
            

In [ ]:
onPolicyMCJC(env, 500, 0.9, 0.1, 100)